Wish to investigate different ways to calculate loop signal. So far I've been computing it by adding the totals and masks from all chromosomes and taking the ratio to get an average pileup. Then I extract the main diagonal that contains the loop and take the ratio of the peak to the outside region. My issue with this technique is that it's one dimensional possibly misrepresents the data which is two dimensional.

Goal 1: Come up with a 2D way of calculating loop signal.
1) Use a edge filter to extract loop and flame region
2) Isolate loop peak
3) Take average of loop to average of outside area
Bonus: Do the same of the flame

Max looked at the loop metrics I calculated using Johan's data and noticed that for HFF plate with PspXI enzyme, the loop signal is ridiculously high. He figures this might be due to the fact that PspXI is an experimental enzyme so it's coverage could be low leading to ratios of two region in the average being high. He suggest taking the ratio of the totals rather than the ratio of averages.

Goal 2: Run both techiniques of calculating loops signals to see if the loop strength changes.

In [312]:
%matplotlib qt5
import dekker
import importlib as imp
imp.reload(dekker)
from scipy.ndimage.filters import gaussian_filter, sobel, median_filter
plt.rcParams["axes.grid"] = False

In [81]:
dek = dekker.DekkerLib()

In [82]:
data = dek.data[dek.data['cell_type']=='HFF plate']

In [250]:
dekker.boxplot(dek.data, 'loop_strength', 'cell_type','enzyme')

{'END': 12, 'ESC': 17, 'HFF beads': 22, 'HFF plate': 25, 'PFE': 7}


/home/sameer/miniconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [83]:
data[data['enzyme']=='PspXI']

,name,cell_type,crosslinker,enzyme,date,flowcell_date,flowcell,Repeat,remarks,genome,...,%internal,error,%error,redundantInteractions,%redundantInteractions,nonRedundantInteractions,%nonRedundantInteractions,comp_sig,TAD_strength,loop_strength
index,,,,,,,,,,,,,,,,,,,,,
70,U54-031116-PspXI-R1__22MAR16_BH7JCHBB_PE50,HFF plate,FA,PspXI,2016-03-11,2016-03-22,BH7JCHBB_PE50,R1,NaN,hg19,...,99.97,338899,0.17,129985271,331.76,69207358,176.64,3.812377,1.302106,65.60667


In [104]:
data = dek.data[dek.data['cell_type']=='HFF plate']
loop_df = dekker.get_loops(data)
loop_df = loop_df.merge(data[['name','cell_type', 'crosslinker', 'enzyme']], on='name', how='outer')

In [106]:
loop_df = loop_df.merge(data[['name','cell_type', 'crosslinker', 'enzyme']], on='name', how='outer')

In [122]:
loop_df.loc[1]

name           U54-HFF-DSG-DpnII-20161118-plate__07DEC16_C-Mo...
mask           [[[460.0, 461.0, 464.0, 463.0, 464.0, 463.0, 4...
totals         [[[366.178864895, 403.79591191, 387.90909623, ...
cell_type                                              HFF plate
crosslinker                                                  DSG
enzyme                                                     DpnII
Name: 1, dtype: object

In [143]:
tot = df['totals'].values[0]
mas = df['mask'].values[0]

In [146]:
print(np.any(np.isnan(tot)), np.any(np.isnan(mas)))
print(np.any(tot==0), np.any(mas==0))

False False
True True


In [271]:
def plot_pileups(df, q= 99.99):

    for i, row in df.iterrows():
        total = np.nansum(row['totals'], axis=0) 
        if len(total.shape) == 0:
            continue
    
        mask = np.nansum(row['mask'], axis=0)
        pileup = total/mask
        f = gaussian_filter(pileup,1)

        fig = plt.figure()
        fig.suptitle('Cell type: '+row['cell_type']+' Enzyme: '+row['enzyme']+' Crosslinker: '+row['crosslinker']
                     +'\n%tran: '+str(row['%trans'])+' loop '+'{:.2f}'.format(row['loop_strength']))
        ax = fig.add_subplot(221)
        ax.set_title('Totals')
        im = ax.imshow(total, vmax=np.percentile(total, q), cmap=matplotlib.cm.get_cmap('plasma'))
        # cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
        # cb = fig.colorbar(im, cax=cax)

        ax = fig.add_subplot(222)
        ax.set_title('Mask')
        im = ax.imshow(mask, vmax=np.percentile(mask, q), cmap=matplotlib.cm.get_cmap('plasma'))
        # cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
        # cb = fig.colorbar(im, cax=cax)

        ax = fig.add_subplot(223)
        ax.set_title('Ratio')
        im = ax.imshow(pileup, vmax=np.percentile(pileup, q), cmap=matplotlib.cm.get_cmap('plasma'))
        cax = fig.add_axes([0.9, 0.2, 0.02, 0.6])
        cb = fig.colorbar(im, cax=cax)

        ax = fig.add_subplot(224)
        ax.set_title('Filtered Ratio')
        im = ax.imshow(f, vmax=np.percentile(f, q), cmap=matplotlib.cm.get_cmap('plasma'))
        # cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
        # cb = fig.colorbar(im, cax=cax)

In [276]:
data = dek.data[dek.data['cell_type']=='HFF plate'].sort_values(by=['enzyme'])
loop_df = dekker.get_loops(data)
loop_df = loop_df.merge(data[['name','cell_type', 'crosslinker', 
                              'enzyme', '%trans', 'loop_strength']], on='name', how='outer')
df = loop_df#[loop_df['enzyme']=='DdeI']
plot_pileups(df)

/home/sameer/miniconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [370]:
df = loop_df[loop_df['enzyme']=='HindIII']
df = df.loc[df.index[0]]
total = np.nansum(df['totals'], axis=0) 
mask = np.nansum(df['mask'], axis=0)
pileup = total/mask
# f = gaussian_filter(total,1)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(total, cmap=matplotlib.cm.get_cmap('plasma'))

mid = len(total)/2-1
span = 8
low = np.floor(mid-span/2).astype(int)
up = np.floor(mid+span/2).astype(int)

region1 = total[0:low,0:low]
inner = total[low:up,low:up]
region2 = total[up:,up:]

mid = len(region1)/2-1
span = 8
low = np.floor(mid-span/2).astype(int)
up = np.floor(mid+span/2).astype(int)
outer = (region1[low:up,low:up]+region2[low:up,low:up])/2
print('Metric 1: ',np.mean(inner/outer))


mid = len(pileup)/2-1
span = 8
low = np.floor(mid-span/2).astype(int)
up = np.floor(mid+span/2).astype(int)

region1 = pileup[0:low,0:low]
inner = pileup[low:up,low:up]
region2 = pileup[up:,up:]

mid = len(region1)/2-1
span = 8
low = np.floor(mid-span/2).astype(int)
up = np.floor(mid+span/2).astype(int)
outer = (region1[low:up,low:up]+region2[low:up,low:up])/2
print('Metric 2: ',np.mean(inner/outer))

Metric 1:  2.6850733419
Metric 2:  2.67107844444


In [371]:
mid = len(total)/2-1
span = 8
low = np.floor(mid-span/2).astype(int)
up = np.floor(mid+span/2).astype(int)

region1 = total[0:low,0:low]
inner = total[low:up,low:up]
region2 = total[up:,up:]

outer = (np.mean(region1)+np.mean(region2))/2
print('Metric 1: ',np.mean(inner)/outer)

mid = len(pileup)/2-1
span = 8
low = np.floor(mid-span/2).astype(int)
up = np.floor(mid+span/2).astype(int)

region1 = pileup[0:low,0:low]
inner = pileup[low:up,low:up]
region2 = pileup[up:,up:]

outer = (np.mean(region1)+np.mean(region2))/2
print('Metric 2: ',np.mean(inner)/outer)

Metric 1:  2.55699316313
Metric 2:  2.54195872903


Still need to do:
1) Average of small sections of region 1,2 for the outer
2) Compare 

2.6276356407243373